## Working withGgeohash in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/10_geohash.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/10_geohash.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/10_geohash.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/10_geohash.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/10_geohash.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2geohash

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2geohash

lat = 10.775276
lon = 106.706797
res = 6
geohash_id = latlon2geohash(lat, lon, res)
geohash_id

### Geohash to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.geohash2geo  import geohash2geo

geohash_geo = geohash2geo(geohash_id)
geohash_geo

### Geohash to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.geohash2geo import geohash2geojson

geohash_geojson = geohash2geojson(geohash_id)
geohash_geojson

### Vector to Geohash

In [ ]:
from vgrid.conversion.vector2dggs.vector2geohash import vector2geohash

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon.geojson"
)
vector_to_geohash = vector2geohash(file_path, resolution=7, compact=False, 
                                    predicate = "intersects",output_format="gpd")
# Visualize the output
vector_to_geohash.plot(edgecolor="white")

### Geohash Compact

In [ ]:
from vgrid.conversion.dggscompact.geohashcompact import geohashcompact

geohash_compacted = geohashcompact(vector_to_geohash, geohash_id="geohash", output_format="gpd")
geohash_compacted.plot(edgecolor="white")


### Geohash Expand

In [ ]:
from vgrid.conversion.dggscompact.geohashcompact import geohashexpand

geohash_expanded = geohashexpand(
    vector_to_geohash, resolution=8, output_format="gpd"        
)
geohash_expanded.plot(edgecolor="white")

### Geohash Binning

In [ ]:
from vgrid.binning.geohashbin import geohashbin
import geopandas as gpd
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "count"
geohash_bin =  geohashbin(file_path, resolution=7, stats=stats, 
                # numeric_field="confidence",
                # category="category",
                output_format="gpd")
geohash_bin.plot(
    column=stats,        # numeric column to base the colors on
    cmap='Spectral_r',        # color scheme (matplotlib colormap)
    legend=True,  
    linewidth=0.2          # boundary width (optional)
)   

### Raster to Geohash

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2geohash    import raster2geohash
from vgrid.utils.io import download_file          

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)

raster_to_geohash =  raster2geohash(raster_file,output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

geohash_layer = folium.GeoJson(
    raster_to_geohash,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 1,
    },
    popup=folium.GeoJsonPopup(
        fields=["geohash", "band_1", "band_2", "band_3"],
        aliases=["Geohash ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(geohash_layer.get_bounds())

# Display the map
m

### Geohash Generator

In [ ]:
from vgrid.generator.geohashgrid import geohashgrid
# geohash_grid = geohashgrid(resolution=1,output_format="gpd")
geohash_grid = geohashgrid(resolution=7,bbox=[106.699007, 10.762811, 106.717674, 10.778649],
                            output_format="gpd")
geohash_grid.plot(edgecolor="white")

### Geohash Inspect

In [ ]:
from vgrid.stats.geohashstats import geohashinspect 
resolution = 3
geohash_inspect = geohashinspect(resolution)    
geohash_inspect.head()

### Distribution of Geohash Area Distortions

In [ ]:
from vgrid.stats.geohashstats import geohash_norm_area
geohash_norm_area(geohash_inspect)

### Distribution of Geohash IPQ Compactness

In [ ]:
from vgrid.stats.geohashstats import geohash_compactness
geohash_compactness(geohash_inspect)

### Geohash Statistics

In [ ]:
from vgrid.stats import geohashstats            
geohashstats()